#  Streamlit CardioRisk App
This notebook defines a simple interactive Streamlit app for predicting cardiovascular risk from clinical input.

In [7]:
import streamlit as st
import xgboost as xgb
import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
import os
import shap
import matplotlib.pyplot as plt

In [8]:
# Load model and feature order
MODEL_PATH = "../src/models/tuned_xgb_fhir.pkl"
FEATURE_PATH = "../src/models/fhir_feature_order.pkl"

booster = joblib.load(MODEL_PATH)
feature_order = joblib.load(FEATURE_PATH)
imputer = SimpleImputer(strategy="mean")


In [9]:
st.title("CardioRisk Estimator")
st.markdown("Predict the Cardio-related risks in 10 years based on clinical biomarkers.")

# Input fields
input_data = {}
for feat in feature_order:
    if feat == "age":
        input_data[feat] = st.number_input("Age", 18, 120, 55)
    elif "BP" in feat:
        input_data[feat] = st.number_input(feat, 50, 250, 120)
    elif "chol" in feat.lower():
        input_data[feat] = st.number_input(feat, 100, 400, 200)
    else:
        input_data[feat] = st.number_input(feat, 30, 300, 100)


2025-05-15 15:12:17.061 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.068 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:17.070 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [12]:
if st.button("Predict"):
    df_input = pd.DataFrame([input_data])[feature_order]
    df_input = pd.DataFrame(imputer.fit_transform(df_input), columns=feature_order)
    dmat = xgb.DMatrix(df_input, feature_names=feature_order)
    pred = booster.predict(dmat)[0]
    st.metric(label="Cardio risk percentage", value=f"{pred:.2%}")

    # SHAP waterfall explanation
    explainer = shap.Explainer(booster)
    shap_values = explainer(df_input)

    st.subheader("Biomarkers atribution")
    fig = shap.plots._waterfall.waterfall_legacy(shap_values[0], show=False)
    st.pyplot(fig)


2025-05-15 15:12:47.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:47.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:47.711 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:47.711 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 15:12:47.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
